# Config Dev

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os, sys
sys.path.insert(0, f"{os.getcwd()}/src")

In [3]:
import torch
from pprint import pprint
import pixiedust

Pixiedust database opened successfully


# Baseline

In [4]:
#checkpoint_file='model1.pt:model2.pt:model3.pt:model4.pt'
path = "/code/huggingface/transformers-fair-wmt"
mname = 'transformer.wmt19.ru-en'
chkpt1_path = path + "/data/wmt19.ru-en.ensemble/model4.pt"
checkpoint_file='model1.pt'

chkpt = torch.load(chkpt1_path, map_location="cpu")

# validate that the model keys are correct:
#hub_interface = torch.hub.load("pytorch/fairseq", mname, checkpoint_file).eval()
#hub_interface.models[0].load_state_dict(chkpt["model"])
#chkpt = hub_interface

# this works just as well, but the above validates that the local copy loads correctly
# chkpt = torch.load(chkpt1_path, map_location="cpu")

In [5]:
#chkpt.models[0].upgrade_state_dict(chkpt.models[0].state_dict())

In [6]:
dir(chkpt)

['__class__', '__contains__', '__delattr__', '__delitem__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__len__', '__lt__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setitem__', '__sizeof__', '__str__', '__subclasshook__', 'clear', 'copy', 'fromkeys', 'get', 'items', 'keys', 'pop', 'popitem', 'setdefault', 'update', 'values']

In [7]:
conf_orig = dict(vars(chkpt["args"]))
#conf_orig
conf_orig["source_lang"]
conf_orig["encoder_embed_dim"]
conf_orig["decoder_embed_dim"]

'ru'

1024

1024

In [8]:
pprint(vars(chkpt["args"]))

{'adam_betas': '(0.9, 0.98)',
 'adam_eps': 1e-08,
 'adaptive_input': False,
 'adaptive_softmax_cutoff': None,
 'adaptive_softmax_dropout': 0,
 'arch': 'transformer_wmt_en_de_big',
 'attention_dropout': 0.1,
 'bucket_cap_mb': 25,
 'clip_norm': 0.0,
 'cpu': False,
 'criterion': 'label_smoothed_cross_entropy',
 'data': ['/private/home/edunov/wmt19/data/old/enru',
          '/private/home/edunov/wmt19/data/old/enru',
          '/private/home/edunov/wmt19/data/finetune/ncru/',
          '/private/home/edunov/wmt19/data/temp/enru'],
 'ddp_backend': 'c10d',
 'decoder_attention_heads': 16,
 'decoder_embed_dim': 1024,
 'decoder_embed_path': None,
 'decoder_ffn_embed_dim': 4096,
 'decoder_input_dim': 1024,
 'decoder_layers': 6,
 'decoder_learned_pos': False,
 'decoder_normalize_before': False,
 'decoder_output_dim': 1024,
 'device_id': 0,
 'distributed_backend': 'nccl',
 'distributed_init_method': 'tcp://localhost:17185',
 'distributed_port': -1,
 'distributed_rank': 0,
 'distributed_world_size'

In [9]:
pprint(vars(chkpt["args"]).keys())

dict_keys(['no_progress_bar', 'log_interval', 'log_format', 'tensorboard_logdir', 'seed', 'cpu', 'fp16', 'memory_efficient_fp16', 'fp16_init_scale', 'fp16_scale_window', 'fp16_scale_tolerance', 'min_loss_scale', 'threshold_loss_scale', 'user_dir', 'task', 'num_workers', 'skip_invalid_size_inputs_valid_test', 'max_tokens', 'max_sentences', 'train_subset', 'valid_subset', 'max_sentences_valid', 'distributed_world_size', 'distributed_rank', 'distributed_backend', 'distributed_init_method', 'distributed_port', 'device_id', 'ddp_backend', 'bucket_cap_mb', 'fix_batches_to_gpus', 'arch', 'criterion', 'max_epoch', 'max_update', 'clip_norm', 'sentence_avg', 'update_freq', 'optimizer', 'lr', 'momentum', 'weight_decay', 'lr_scheduler', 'lr_shrink', 'min_lr', 'save_dir', 'restore_file', 'reset_optimizer', 'reset_lr_scheduler', 'optimizer_overrides', 'save_interval', 'save_interval_updates', 'keep_interval_updates', 'keep_last_epochs', 'no_save', 'no_epoch_checkpoints', 'validate_interval', 'no_tok

In [10]:
from transformers.modeling_fsmt import FSMTForConditionalGeneration
from transformers.configuration_fsmt import FSMTConfig

In [11]:

hf_checkpoint_name = "/code/huggingface/transformers-fair-wmt/data/wmt19-ru-en/config.json"
config = FSMTConfig.from_pretrained(hf_checkpoint_name)
state_dict = chkpt["model"]

In [12]:
#chkpt["model"].keys()

In [13]:
# renames/removal
from collections import OrderedDict

rename_keys = [
#    ("model.encoder.embed_positions._float_tensor", "model.encoder.embed_positions.weight"),
#    ("model.decoder.embed_positions._float_tensor", "model.decoder.embed_positions.weight"),
#    ("", ""),
#    ("", ""),
#    ("", ""),
#    ("", ""),    
]

def remove_ignore_keys_(state_dict):
    ignore_keys = [
        "model.model",
        "model.encoder.version",
        "model.decoder.version",
        "model.encoder.embed_positions._float_tensor", # not storing model.encoder.embed_positions.weight
        "model.decoder.embed_positions._float_tensor", # not storing model.decoder.embed_positions.weight
    ]
    for k in ignore_keys:
        state_dict.pop(k, None)

def rename_key(dct, old, new):
    val = dct.pop(old)
    dct[new] = val

#state_dict = chkpt["model"].copy()

# rename keys to start with model.
state_dict = OrderedDict(("model."+k, v) for k, v in chkpt["model"].items())
# check:
#state_dict["model.encoder.layers.0.fc1.bias"]
#chkpt["model"]["encoder.layers.0.fc1.bias"]
    
remove_ignore_keys_(state_dict)
for src, dest in rename_keys:
    rename_key(state_dict, src, dest)

In [14]:
#pprint(state_dict.keys())
#state_dict["model.encoder.layers.0.fc1.bias"]
#state_dict["model.encoder.embed_positions.weight"]
#torch.FloatTensor(1)

In [15]:
#%%pixie_debugger -b /code/huggingface/transformers-fair-wmt/src/transformers/modeling_fsmt.py:913

model = FSMTForConditionalGeneration(config).eval()
# show missing or extraneous/mismatching keys (need to remap/change model to match)
model.load_state_dict(state_dict)

1024 1024 1


RuntimeError: Error(s) in loading state_dict for FSMTForConditionalGeneration:
	Missing key(s) in state_dict: "final_logits_bias", "model.encoder_embed_tokens.weight", "model.decoder_embed_tokens.weight", "model.encoder.embed_positions.weight", "model.encoder.layers.0.self_attn.k_proj.weight", "model.encoder.layers.0.self_attn.k_proj.bias", "model.encoder.layers.0.self_attn.v_proj.weight", "model.encoder.layers.0.self_attn.v_proj.bias", "model.encoder.layers.0.self_attn.q_proj.weight", "model.encoder.layers.0.self_attn.q_proj.bias", "model.encoder.layers.0.self_attn_layer_norm.weight", "model.encoder.layers.0.self_attn_layer_norm.bias", "model.encoder.layers.0.final_layer_norm.weight", "model.encoder.layers.0.final_layer_norm.bias", "model.encoder.layers.1.self_attn.k_proj.weight", "model.encoder.layers.1.self_attn.k_proj.bias", "model.encoder.layers.1.self_attn.v_proj.weight", "model.encoder.layers.1.self_attn.v_proj.bias", "model.encoder.layers.1.self_attn.q_proj.weight", "model.enco

In [ ]:
state_dict.keys()

In [ ]:
from torch import nn
emb = nn.Embedding(2, 1, 0)
#t1 = emb(torch.LongTensor([0]))
#t1.shape
emb.num_embeddings

# Target

In [ ]:
from transformers.tokenization_fsmt import FSMTTokenizer
from transformers.modeling_fsmt import FSMTModel

In [ ]:
"Success!!!"

In [ ]:
a = b = 1

In [ ]:
import re
authorized_missing_keys = [r"final_logits_bias", r"encoder\.version", r"decoder\.version"]
missing_keys = ["final_logits_bias"]
for pat in authorized_missing_keys:
    missing_keys = [k for k in missing_keys if re.search(pat, k) is None]
missing_keys